**라이브러리 가져오기 / 데이터 정리 **

In [ ]:
from rank_bm25 import BM25Okapi
import pandas as pd
from tqdm import tqdm

In [ ]:
train = pd.read_csv("train.csv", encoding="utf8")
questions = pd.read_csv("question.csv", encoding="utf8")
collections = pd.read_csv("collection.csv", encoding="utf8")

** 모델 학습 **

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
class BM25Ranker1:
    def __init__(self, documents):
        self.documents = documents
        self.tokenized_documents = [doc.split() for doc in documents]
        self.bm25 = BM25Okapi(self.tokenized_documents, k1= 1.4)

    def rank_documents(self, query, top_k):
        tokenized_query = query.split()
        scores = self.bm25.get_scores(tokenized_query)
        ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
        ranked_documents = [self.documents[i] for i in ranked_indices]
        return ranked_documents

    def calculate_recall(self, queries, answers, top_k_values):
        total_queries = len(queries)
        recall_values = {k: 0 for k in top_k_values}

        with tqdm(total=total_queries, desc="Calculating Recall") as pbar:
            for query, answer in zip(queries, answers):
                ranked_documents = self.rank_documents(query, max(top_k_values))
                relevant_documents = [document for document in ranked_documents if document == answer]

                for k in top_k_values:
                    if answer in ranked_documents[:k]:
                        recall_values[k] += 1
                    else:
                        if k == 100:
                            print("query: ", query)
                            print("answer: ", answer)
                            print("the first rank: ", ranked_documents[0])

                pbar.update(1)

        recall_values = {k: v / total_queries for k, v in recall_values.items()}
        return recall_values

In [ ]:
questions.columns = ['0', 'x_id', 'content']
collections.columns = ['0', 'y_id', 'document']

train = train.merge(questions, on="x_id", how="left")

train.dropna(inplace = True)
train.reset_index(inplace = True, drop=True)

train = train.merge(collections, on="y_id", how="left")
train.dropna(inplace=True)
train.reset_index(inplace=True, drop=True)

In [ ]:
train = train[:10000]

In [ ]:
documents = list(collections['document'])
queries = list(train['content'])
answers = list(train['document'])

In [ ]:
ranker = BM25Ranker1(documents)

In [ ]:
recall_values = ranker.calculate_recall(queries, answers, [1, 10, 20, 100])
print(recall_values)

In [ ]:
keys = d.keys()
keys

In [ ]:
query = "인수 공통감염병 관리 전략 워크숍이 시행되는 장소는 어디니"
query = query.split()
print(query)
print(ranker.bm25.get_scores(query))
tfidf_scores = ranker.bm25.get_scores(query, b=0.0)
print(tfidf_scores)

In [ ]:
len(ranker.bm25.get_scores(query))

In [ ]:
query = "원헬스적 접근을 통한 인수공통감염병 관리전략 워크숍 계획  행사개요 ○ 행사일시 : 2021. 4. 1.(목) 15:00~17:00 ○ 행사장소 : 시청 7층 공용회의실 ○ 행사방법 : 필수인원 현장 참석, 관심직원 온라인 참여 동시 진행 ○ 참석대상 - (오프라인) 시민건강국장, 보건의료정책과, 감염병관리과, 식품정책과, 동물보호과, 감염병연구센터, 보건환경연구원 동물위생시험소 ※ 코로나 19 예방행동수칙(회의수칙) 준수 - (온라인) 자치구, 보건환경연구원, 서울대공원 등 관심있는 직원 ※ 사전 온라인 참여 신청을 받은 직원에게 당일 초대링크 발송 ○ 주요내용 : 국내외 원헬스 동향 및 원헬스적 접근을 통한 인수공통감염병 관리 전략 마련 ○ 강사 : *** 교수(서울대학교 수의과대학, 원헬스의 이론과 실제), ** 교수(동국대학교 의과대학, 인수공통감염병 관리체계 구축)  소요예산 ○ 산출금액 : 480천원 - 강사료 : 240,000원 × 1시간 × 2인 = 480,000원 ○ 예산과목 : 선진형 동물보호시스템 구축, 수의공중보건 강화, 가축방역-예방 약품 등 지원(자체), 일반운영비, 사무관리비.  행정사항 ○ 교육참석자 교육시간(2시간) 인정."
query = query.split()
print(query)
score = ranker.bm25.get_scores(query)

max_index = np.argmax(score)

# Get the maximum value using the index
max_value = score[max_index]

query[max_index]
max_value

In [ ]:
query

In [ ]:
len(score)

In [ ]:
max_value = max(score)

# Get the index of the maximum value
max_index = score.index(max_value)

query[max_index]
max_value

In [ ]:
query = "제목 대여유물 인수 결과보고(국립현대미술관) 1. 국립현대미술관 <미술이 문학을 만났을 때> 전시에 대여했던 우리 박물관 소장유물을  다음과 같이 인수하였기에 결과를 보고합니다. 가. 유물 내역 : ******************** 나. 대여 기간 : 2021년 1월 22일 ~ 2021년 6월 2일 다. 인수 인계일 : 2021년 6월 2일 라. 인수 인계자 - 인계자 : 국립현대미술관 *** - 인수자 : 서울역사박물관 *** 마. 이상 유무 : 이상 없음 / 확인자 : 유물관리과 *** "
query = query.split()
print(query)
ranker.bm25.get_scores(query)

In [ ]:
import json

with open("BM25_result8.json", "w", encoding="utf8") as json_file:
    json.dump(recall_values, json_file, ensure_ascii=False)

In [ ]:
class BM25Ranker2:
    def __init__(self, documents):
        self.documents = documents
        self.tokenized_documents = [doc.split() for doc in documents]
        self.bm25 = BM25Okapi(self.tokenized_documents, k1=1.6)

    def rank_documents(self, query, top_k):
        tokenized_query = query.split()
        scores = self.bm25.get_scores(tokenized_query)
        ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
        ranked_documents = [self.documents[i] for i in ranked_indices]
        return ranked_documents

    def calculate_recall(self, queries, answers, top_k_values):
        total_queries = len(queries)
        recall_values = {k: 0 for k in top_k_values}

        with tqdm(total=total_queries, desc="Calculating Recall") as pbar:
            for query, answer in zip(queries, answers):
                ranked_documents = self.rank_documents(query, max(top_k_values))
                relevant_documents = [document for document in ranked_documents if document == answer]

                for k in top_k_values:
                    if answer in ranked_documents[:k]:
                        recall_values[k] += 1

                pbar.update(1)

        recall_values = {k: v / total_queries for k, v in recall_values.items()}
        return recall_values
    
ranker1 = BM25Ranker2(documents)
recall_values1 = ranker1.calculate_recall(queries, answers, [1, 10, 20, 100])
print(recall_values1)
with open("BM25_result6.json", "w", encoding="utf8") as json_file:
    json.dump(recall_values1, json_file, ensure_ascii=False)

In [ ]:
class BM25Ranker3:
    def __init__(self, documents):
        self.documents = documents
        self.tokenized_documents = [doc.split() for doc in documents]
        self.bm25 = BM25Okapi(self.tokenized_documents, k1=1.8)

    def rank_documents(self, query, top_k):
        tokenized_query = query.split()
        scores = self.bm25.get_scores(tokenized_query)
        ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
        ranked_documents = [self.documents[i] for i in ranked_indices]
        return ranked_documents

    def calculate_recall(self, queries, answers, top_k_values):
        total_queries = len(queries)
        recall_values = {k: 0 for k in top_k_values}

        with tqdm(total=total_queries, desc="Calculating Recall") as pbar:
            for query, answer in zip(queries, answers):
                ranked_documents = self.rank_documents(query, max(top_k_values))
                relevant_documents = [document for document in ranked_documents if document == answer]

                for k in top_k_values:
                    if answer in ranked_documents[:k]:
                        recall_values[k] += 1

                pbar.update(1)

        recall_values = {k: v / total_queries for k, v in recall_values.items()}
        return recall_values
    
ranker2 = BM25Ranker3(documents)
recall_values2 = ranker2.calculate_recall(queries, answers, [1, 10, 20, 100])
print(recall_values2)
with open("BM25_result7.json", "w", encoding="utf8") as json_file:
    json.dump(recall_values2, json_file, ensure_ascii=False)